In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([False,  True, False,  ...,  True, False, False])

In [2]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [3]:
get_parallel_db(db, 52352)

tensor([False,  True, False,  ...,  True, False, False])

In [5]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [6]:
pdbs = get_parallel_dbs(db)

In [7]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [8]:
db, pdbs = create_db_and_parallels(20)

In [9]:
db, pdbs = create_db_and_parallels(5000)

In [10]:
def query(db):
    return db.sum()

In [11]:
full_db_result = query(db)

In [12]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [13]:
sensitivity

tensor(1)

In [19]:
def sensitivity(query, n_entries=1000):

    db, pdbs = create_db_and_parallels(n_entries)
    
    print(db.sum())
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
            
    return max_distance

In [20]:
def query(db):
    return db.float().mean()

In [21]:
sensitivity(query)

tensor(520)


tensor(0.0005)

In [17]:
db, pdbs = create_db_and_parallels(20)

In [18]:
db

tensor([False,  True, False, False, False,  True,  True, False, False, False,
         True,  True, False, False, False, False, False, False,  True,  True])

In [22]:
519/999-520/1000


-0.0004804804804805496

In [23]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()

In [27]:
for i in range(10):
    sens_f = sensitivity(query, n_entries=10)
    print(sens_f)

tensor(6)
tensor(1.)
tensor(9)
0
tensor(3)
0
tensor(3)
0
tensor(6)
tensor(1.)
tensor(6)
tensor(1.)
tensor(6)
tensor(1.)
tensor(3)
0
tensor(7)
0
tensor(3)
0


In [28]:
db, _ = create_db_and_parallels(100)

In [29]:
pdb = get_parallel_db(db, remove_index=10)

In [30]:
db[10]

tensor(False)

In [31]:
sum(db)

tensor(47)

In [32]:
sum(db) - sum(pdb)

tensor(0)

In [33]:
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0047)

In [36]:
(sum(db).float() > 49) ^ (sum(pdb).float()  > 49)

tensor(False)

In [37]:
db

tensor([False, False,  True, False,  True, False,  True,  True, False,  True,
        False,  True, False,  True,  True,  True,  True,  True, False, False,
         True, False, False, False,  True, False, False, False,  True,  True,
        False, False,  True, False, False,  True,  True, False, False, False,
        False,  True,  True,  True, False, False,  True, False,  True,  True,
        False, False, False, False, False,  True, False, False, False,  True,
         True, False, False,  True, False,  True, False, False,  True, False,
        False,  True,  True,  True,  True,  True,  True, False,  True, False,
         True,  True,  True, False, False,  True,  True, False,  True,  True,
        False,  True, False,  True, False, False, False, False,  True, False])

In [38]:
def query(db):

    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [39]:
num=1
for i in range(5):
    num*=10
    db, pdbs = create_db_and_parallels(num)
    private_result, true_result = query(db)
    print("With Noise:" + str(private_result))
    print("Without Noise:" + str(true_result))

With Noise:tensor(0.5000)
Without Noise:tensor(0.4000)


In [40]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.7600)
Without Noise:tensor(0.5000)
